In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
cd ~/demres

/Users/zurfarosa/demres


In [3]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.tools.tools import add_constant
from statsmodels.stats.outliers_influence import variance_inflation_factor 
import pylab as pl
from datetime import date, timedelta

import demres
from demres.common.constants import entry_type
from demres.demins.constants import Study_Design as sd
from demres.common import codelists
from demres.common.helper_functions import *
from demres.demins.statistical_functions import *

In [4]:
pd.set_option('display.max_columns', None)

## Specifiy dementia subtype

In [5]:
subtype = 'all_dementia' # options: 'alzheimers', 'vascular','all_dementia'

## Specify exposure window

In [6]:
window = '8_to_3' #options: '12_to_7','10_to_5','8_to_3'

## Load relevant dataframe and create intercept

In [7]:
pt_features = pd.read_csv('data/pt_data/processed_data/pt_features_demins_' + subtype + '_' + window +'.csv',delimiter=',',parse_dates=['index_date','data_end','data_start'],infer_datetime_format=True)

In [8]:
pt_features.columns

Index(['patid', 'yob', 'pracid', 'female', 'index_date', 'isCase',
       'final dementia medcode', 'data_end', 'data_start', 'matchid',
       'reason_for_removal', 'age_at_index_date', 'insomnia', 'stroke',
       'non_stroke_vascular_disease', 'hypertension', 'diabetes',
       'mental_illness_non_smi', 'mental_illness_smi', 'sleep_apnoea',
       'chronic_pulmonary_disease', 'epilepsy', 'mood_stabilisers_100_pdds',
       'benzo_and_z_drugs_100_pdds', 'other_sedatives_100_pdds',
       'antipsychotics_100_pdds', 'depot_antipsychotics_100_pdds',
       'antidepressants_100_pdds', 'non_insomnia_GP_consultations',
       'benzo_and_z_drugs_any', 'insomnia_any', 'insomnia_count:0',
       'insomnia_count:1_5', 'insomnia_count:above_5',
       'non_insomnia_GP_consultations:0', 'non_insomnia_GP_consultations:1_10',
       'non_insomnia_GP_consultations:11_100',
       'non_insomnia_GP_consultations:101_1000',
       'non_insomnia_GP_consultations:above_1000', 'age_at_index_date:65-69',


In [9]:
pt_features.loc['intercept'] = 1.0

In [10]:
pt_features.sample(5)

,patid,yob,pracid,female,index_date,isCase,final dementia medcode,data_end,data_start,matchid,reason_for_removal,age_at_index_date,insomnia,stroke,non_stroke_vascular_disease,hypertension,diabetes,mental_illness_non_smi,mental_illness_smi,sleep_apnoea,chronic_pulmonary_disease,epilepsy,mood_stabilisers_100_pdds,benzo_and_z_drugs_100_pdds,other_sedatives_100_pdds,antipsychotics_100_pdds,depot_antipsychotics_100_pdds,antidepressants_100_pdds,non_insomnia_GP_consultations,benzo_and_z_drugs_any,insomnia_any,insomnia_count:0,insomnia_count:1_5,insomnia_count:above_5,non_insomnia_GP_consultations:0,non_insomnia_GP_consultations:1_10,non_insomnia_GP_consultations:11_100,non_insomnia_GP_consultations:101_1000,non_insomnia_GP_consultations:above_1000,age_at_index_date:65-69,age_at_index_date:70-74,age_at_index_date:75-79,age_at_index_date:80-84,age_at_index_date:85-89,age_at_index_date:90-99,age_at_index_date:above_99,antidepressant_pdds:00000,antidepressant_pdds:00001_10,antidepressant_pdds:00011_100,antidepressant_pdds:00101_1000,antidepressant_pdds:01001_10000,antidepressant_pdds:10000_and_above,antipsychotic_pdds:00000,antipsychotic_pdds:00001_10,antipsychotic_pdds:00011_100,antipsychotic_pdds:00101_1000,antipsychotic_pdds:01001_10000,antipsychotic_pdds:10000_and_above,depot_antipsychotic_pdds:00000,depot_antipsychotic_pdds:00001_10,depot_antipsychotic_pdds:00011_100,depot_antipsychotic_pdds:00101_1000,depot_antipsychotic_pdds:01001_10000,depot_antipsychotic_pdds:10000_and_above,other_sedative_pdds:00000,other_sedative_pdds:00001_10,other_sedative_pdds:00011_100,other_sedative_pdds:00101_1000,other_sedative_pdds:01001_10000,other_sedative_pdds:10000_and_above,benzo_and_z_drug_pdds:00000,benzo_and_z_drug_pdds:00001_10,benzo_and_z_drug_pdds:00011_100,benzo_and_z_drug_pdds:00101_1000,benzo_and_z_drug_pdds:01001_10000,benzo_and_z_drug_pdds:10000_and_above,mood_stabiliser_pdds:00000,mood_stabiliser_pdds:00001_10,mood_stabiliser_pdds:00011_100,mood_stabiliser_pdds:00101_1000,mood_stabiliser_pdds:01001_10000,mood_stabiliser_pdds:10000_and_above
23636,10792211.0,26.0,211.0,0.0,2002-02-01 00:00:00,0.0,NaN,2006-11-13 00:00:00,1993-01-31 00:00:00,25898.0,NaN,76.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,0.0,0.0,31.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
42040,2733320.0,24.0,320.0,1.0,2009-06-12 00:00:00,1.0,6578.0,2009-08-12 00:00:00,1990-01-31 00:00:00,48466.0,NaN,85.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.71443,0.0,0.129284,0.0,0.0,161.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
33042,3586143.0,13.0,143.0,0.0,2005-07-28 00:00:00,1.0,11175.0,2008-09-22 00:00:00,1993-01-31 00:00:00,21110.0,NaN,92.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,0.0,0.0,115.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4967,9763402.0,21.0,402.0,1.0,2011-01-26 00:00:00,1.0,4693.0,2011-06-27 00:00:00,2000-08-22 00:00:00,59615.0,NaN,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,0.0,0.0,56.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
20039,2739084.0,21.0,84.0,1.0,2010-02-19 00:00:00,0.0,NaN,2013-04-10 00:00:00,1999-12-10 00:00:00,13821.0,NaN,89.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,0.0,0.0,32.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.

In [11]:
columns_always_for_inclusion = [
    'age_at_index_date',
    'female',
#      'non_insomnia_GP_consultations', removed as VIF 4.3 (more importantly, age at index date is 5.7 in its presence)
     'stroke',
    'non_stroke_vascular_disease',
    'hypertension',
    'diabetes', 
    'mental_illness_non_smi', 
    'mental_illness_smi',
    'sleep_apnoea', 'chronic_pulmonary_disease', 'epilepsy',
    'mood_stabilisers_100_pdds',
    'other_sedatives_100_pdds', 'antidepressants_100_pdds', 
    'depot_antipsychotics_100_pdds', 'antipsychotics_100_pdds',
    'benzo_and_z_drugs_100_pdds'
]

## Insomnia consultations

### insomnia yes/no

In [12]:
cols_for_inclusion_here = ['insomnia_any','benzo_and_z_drugs_100_pdds']
training_cols = [col for col in pt_features.columns if col in cols_for_inclusion_here+columns_always_for_inclusion]
summary_table,univariate_and_multivariate_results,univariate_and_multivariate_results_formatted = purposefully_select_covariates(pt_features,covariates=training_cols,main_variables=cols_for_inclusion_here)

female  being retained as mean > 0
age_at_index_date  being retained as mean > 0
stroke  being retained as mean > 0
non_stroke_vascular_disease  being retained as mean > 0
hypertension  being retained as mean > 0
diabetes  being retained as mean > 0
mental_illness_non_smi  being retained as mean > 0
mental_illness_smi  being retained as mean > 0
sleep_apnoea  being retained as mean > 0
chronic_pulmonary_disease  being retained as mean > 0
epilepsy  being retained as mean > 0
mood_stabilisers_100_pdds  being retained as mean > 0
benzo_and_z_drugs_100_pdds  being retained as mean > 0
other_sedatives_100_pdds  being retained as mean > 0
antipsychotics_100_pdds  being retained as mean > 0
depot_antipsychotics_100_pdds  being retained as mean > 0
antidepressants_100_pdds  being retained as mean > 0
insomnia_any  being retained as mean > 0


*Stage 1*
Univariate results
                               Univariate OR  [0.025  0.975]  p value
age_at_index_date                       1.00    1.00 

In [13]:
univariate_and_multivariate_results

,Univariate OR,[0.025,0.975],p value,Multivariate OR,multi [0.025,multi 0.975],multi p value
age_at_index_date,1.00,1.00,1.00,1.0,1.00,1.00,1.00,0.000
antidepressants_100_pdds,1.04,1.03,1.04,0.0,1.02,1.01,1.02,0.000
benzo_and_z_drugs_100_pdds,1.02,1.02,1.03,0.0,1.01,1.00,1.01,0.000
chronic_pulmonary_disease,1.35,1.29,1.41,0.0,1.32,1.26,1.39,0.000
diabetes,1.39,1.30,1.48,0.0,1.34,1.25,1.43,0.000
epilepsy,1.71,1.45,2.01,0.0,1.42,1.20,1.69,0.000
hypertension,1.08,1.05,1.11,0.0,1.07,1.03,1.11,0.001
insomnia_any,1.51,1.37,1.66,0.0,1.22,1.10,1.35,0.000
mental_illness_non_smi,1.65,1.59,1.71,0.0,1.77,1.70,1.86,0.000
mood_stabilisers_100_pdds,1.04,1.02,1.05,0.0,1.02,1.00,1.03,0.006


In [14]:
univariate_and_multivariate_results_formatted

,Univariate OR,Multivariate OR
Age at index date,"1.00 (95% CI=1.00-1.00, p=1.00)","1.00 (95% CI=1.00-1.00, p=0.00)"
Antidepressants (100 PDDs),"1.04 (95% CI=1.03-1.04, p=0.00)","1.02 (95% CI=1.01-1.02, p=0.00)"
Benzodiazepines and z-drugs (100 PDDs),"1.02 (95% CI=1.02-1.03, p=0.00)","1.01 (95% CI=1.00-1.01, p=0.00)"
Chronic pulmonary disease,"1.35 (95% CI=1.29-1.41, p=0.00)","1.32 (95% CI=1.26-1.39, p=0.00)"
Diabetes,"1.39 (95% CI=1.30-1.48, p=0.00)","1.34 (95% CI=1.25-1.43, p=0.00)"
Epilepsy,"1.71 (95% CI=1.45-2.01, p=0.00)","1.42 (95% CI=1.20-1.69, p=0.00)"
Hypertension,"1.08 (95% CI=1.05-1.11, p=0.00)","1.07 (95% CI=1.03-1.11, p=0.00)"
Insomnia any,"1.51 (95% CI=1.37-1.66, p=0.00)","1.22 (95% CI=1.10-1.35, p=0.00)"
Mental illness non smi,"1.65 (95% CI=1.59-1.71, p=0.00)","1.77 (95% CI=1.70-1.86, p=0.00)"
Mood stabilisers (100 PDDs),"1.04 (95% CI=1.02-1.05, p=0.00)","1.02 (95% CI=1.00-1.03, p=0.01)"


### insomnia (continuous variable)

In [15]:
# cols_for_inclusion_here = ['insomnia','benzo_and_z_drugs_100_pdds']
# training_cols = [col for col in pt_features.columns if col in cols_for_inclusion_here+columns_always_for_inclusion]
# summary_table,univariate_and_multivariate_results,univariate_and_multivariate_results_formatted = purposefully_select_covariates(pt_features,covariates=training_cols,main_variables=cols_for_inclusion_here)

In [16]:
# univariate_and_multivariate_results

### insomnia quantiles

In [17]:
cols_for_inclusion_here = ['insomnia_count:1_5','insomnia_count:above_5','benzo_and_z_drugs_100_pdds']
training_cols = [col for col in pt_features.columns if col in cols_for_inclusion_here+columns_always_for_inclusion]
summary_table,univariate_and_multivariate_results,univariate_and_multivariate_results_formatted = purposefully_select_covariates(pt_features,covariates=training_cols,main_variables=cols_for_inclusion_here)

female  being retained as mean > 0
age_at_index_date  being retained as mean > 0
stroke  being retained as mean > 0
non_stroke_vascular_disease  being retained as mean > 0
hypertension  being retained as mean > 0
diabetes  being retained as mean > 0
mental_illness_non_smi  being retained as mean > 0
mental_illness_smi  being retained as mean > 0
sleep_apnoea  being retained as mean > 0
chronic_pulmonary_disease  being retained as mean > 0
epilepsy  being retained as mean > 0
mood_stabilisers_100_pdds  being retained as mean > 0
benzo_and_z_drugs_100_pdds  being retained as mean > 0
other_sedatives_100_pdds  being retained as mean > 0
antipsychotics_100_pdds  being retained as mean > 0
depot_antipsychotics_100_pdds  being retained as mean > 0
antidepressants_100_pdds  being retained as mean > 0
insomnia_count:1_5  being retained as mean > 0
insomnia_count:above_5  being retained as mean > 0


*Stage 1*
Univariate results
                               Univariate OR  [0.025  0.975]  p va

In [18]:
univariate_and_multivariate_results

,Univariate OR,[0.025,0.975],p value,Multivariate OR,multi [0.025,multi 0.975],multi p value
age_at_index_date,1.00,1.00,1.00,1.0,1.00,1.00,1.00,0.000
antidepressants_100_pdds,1.04,1.03,1.04,0.0,1.02,1.01,1.02,0.000
benzo_and_z_drugs_100_pdds,1.02,1.02,1.03,0.0,1.01,1.00,1.01,0.000
chronic_pulmonary_disease,1.35,1.29,1.41,0.0,1.32,1.25,1.39,0.000
diabetes,1.39,1.30,1.48,0.0,1.34,1.25,1.43,0.000
epilepsy,1.71,1.45,2.01,0.0,1.42,1.20,1.69,0.000
hypertension,1.08,1.05,1.11,0.0,1.07,1.03,1.11,0.001
insomnia_count:1_5,1.48,1.34,1.63,0.0,1.20,1.08,1.33,0.001
insomnia_count:above_5,3.25,1.70,6.21,0.0,2.21,1.14,4.29,0.019
mental_illness_non_smi,1.65,1.59,1.71,0.0,1.77,1.70,1.86,0.000


In [19]:
univariate_and_multivariate_results_formatted

,Univariate OR,Multivariate OR
Age at index date,"1.00 (95% CI=1.00-1.00, p=1.00)","1.00 (95% CI=1.00-1.00, p=0.00)"
Antidepressants (100 PDDs),"1.04 (95% CI=1.03-1.04, p=0.00)","1.02 (95% CI=1.01-1.02, p=0.00)"
Benzodiazepines and z-drugs (100 PDDs),"1.02 (95% CI=1.02-1.03, p=0.00)","1.01 (95% CI=1.00-1.01, p=0.00)"
Chronic pulmonary disease,"1.35 (95% CI=1.29-1.41, p=0.00)","1.32 (95% CI=1.25-1.39, p=0.00)"
Diabetes,"1.39 (95% CI=1.30-1.48, p=0.00)","1.34 (95% CI=1.25-1.43, p=0.00)"
Epilepsy,"1.71 (95% CI=1.45-2.01, p=0.00)","1.42 (95% CI=1.20-1.69, p=0.00)"
Hypertension,"1.08 (95% CI=1.05-1.11, p=0.00)","1.07 (95% CI=1.03-1.11, p=0.00)"
Insomnia count:1 5,"1.48 (95% CI=1.34-1.63, p=0.00)","1.20 (95% CI=1.08-1.33, p=0.00)"
Insomnia count:above 5,"3.25 (95% CI=1.70-6.21, p=0.00)","2.21 (95% CI=1.14-4.29, p=0.02)"
Mental illness non smi,"1.65 (95% CI=1.59-1.71, p=0.00)","1.77 (95% CI=1.70-1.86, p=0.00)"


***

***

# Females only

In [20]:
female_pt_features = pt_features[pt_features['female']==True]

In [21]:
female_pt_features

,patid,yob,pracid,female,index_date,isCase,final dementia medcode,data_end,data_start,matchid,reason_for_removal,age_at_index_date,insomnia,stroke,non_stroke_vascular_disease,hypertension,diabetes,mental_illness_non_smi,mental_illness_smi,sleep_apnoea,chronic_pulmonary_disease,epilepsy,mood_stabilisers_100_pdds,benzo_and_z_drugs_100_pdds,other_sedatives_100_pdds,antipsychotics_100_pdds,depot_antipsychotics_100_pdds,antidepressants_100_pdds,non_insomnia_GP_consultations,benzo_and_z_drugs_any,insomnia_any,insomnia_count:0,insomnia_count:1_5,insomnia_count:above_5,non_insomnia_GP_consultations:0,non_insomnia_GP_consultations:1_10,non_insomnia_GP_consultations:11_100,non_insomnia_GP_consultations:101_1000,non_insomnia_GP_consultations:above_1000,age_at_index_date:65-69,age_at_index_date:70-74,age_at_index_date:75-79,age_at_index_date:80-84,age_at_index_date:85-89,age_at_index_date:90-99,age_at_index_date:above_99,antidepressant_pdds:00000,antidepressant_pdds:00001_10,antidepressant_pdds:00011_100,antidepressant_pdds:00101_1000,antidepressant_pdds:01001_10000,antidepressant_pdds:10000_and_above,antipsychotic_pdds:00000,antipsychotic_pdds:00001_10,antipsychotic_pdds:00011_100,antipsychotic_pdds:00101_1000,antipsychotic_pdds:01001_10000,antipsychotic_pdds:10000_and_above,depot_antipsychotic_pdds:00000,depot_antipsychotic_pdds:00001_10,depot_antipsychotic_pdds:00011_100,depot_antipsychotic_pdds:00101_1000,depot_antipsychotic_pdds:01001_10000,depot_antipsychotic_pdds:10000_and_above,other_sedative_pdds:00000,other_sedative_pdds:00001_10,other_sedative_pdds:00011_100,other_sedative_pdds:00101_1000,other_sedative_pdds:01001_10000,other_sedative_pdds:10000_and_above,benzo_and_z_drug_pdds:00000,benzo_and_z_drug_pdds:00001_10,benzo_and_z_drug_pdds:00011_100,benzo_and_z_drug_pdds:00101_1000,benzo_and_z_drug_pdds:01001_10000,benzo_and_z_drug_pdds:10000_and_above,mood_stabiliser_pdds:00000,mood_stabiliser_pdds:00001_10,mood_stabiliser_pdds:00011_100,mood_stabiliser_pdds:00101_1000,mood_stabiliser_pdds:01001_10000,mood_stabiliser_pdds:10000_and_above
1,6120339.0,25.0,339.0,1.0,2000-12-12 00:00:00,1.0,4693.0,2000-12-18 00:00:00,1992-11-30 00:00:00,51222.0,NaN,75.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,35.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,3952335.0,22.0,335.0,1.0,2002-10-01 00:00:00,1.0,1917.0,2002-10-21 00:00:00,1994-09-30 00:00:00,50657.0,NaN,80.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,1.766887,0.0,0.000000,73.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,4688370.0,22.0,370.0,1.0,2007-09-21 00:00:00,1.0,6578.0,2007-10-11 00:00:00,1999-09-15 00:00:00,55106.0,NaN,85.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,53.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,14416079.0,16.0,79.0,1.0,2006-08-07 00:00:00,1.0,4693.0,2006-08-16 00:00:00,1998-07-19 00:00:00,10477.0,NaN,90.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,13.412302,0.000000,0.000000,0.0,0.000000,42.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
6,465452.0,18.0,452.0,1.0,2006-03-08 00:00:00,1.0,1916.0,2006-03-08 00:00:00,1998-01-31 00:00:00,66898.0,NaN,88.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,39.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.

In [22]:
columns_always_for_inclusion_female = [col for col in columns_always_for_inclusion if col != 'female']

### insomnia quantiles

In [23]:
cols_for_inclusion_here = ['insomnia_count:1_5','insomnia_count:above_5','benzo_and_z_drugs_100_pdds']
training_cols = [col for col in female_pt_features.columns if col in cols_for_inclusion_here+columns_always_for_inclusion_female]
summary_table,univariate_and_multivariate_results,univariate_and_multivariate_results_formatted = purposefully_select_covariates(female_pt_features,covariates=training_cols,main_variables=cols_for_inclusion_here)

age_at_index_date  being retained as mean > 0
stroke  being retained as mean > 0
non_stroke_vascular_disease  being retained as mean > 0
hypertension  being retained as mean > 0
diabetes  being retained as mean > 0
mental_illness_non_smi  being retained as mean > 0
mental_illness_smi  being retained as mean > 0
sleep_apnoea  being retained as mean > 0
chronic_pulmonary_disease  being retained as mean > 0
epilepsy  being retained as mean > 0
mood_stabilisers_100_pdds  being retained as mean > 0
benzo_and_z_drugs_100_pdds  being retained as mean > 0
other_sedatives_100_pdds  being retained as mean > 0
antipsychotics_100_pdds  being retained as mean > 0
depot_antipsychotics_100_pdds  being retained as mean > 0
antidepressants_100_pdds  being retained as mean > 0
insomnia_count:1_5  being retained as mean > 0
insomnia_count:above_5  being retained as mean > 0


*Stage 1*
Univariate results
                               Univariate OR  [0.025  0.975]  p value
age_at_index_date              

In [24]:
univariate_and_multivariate_results

,Univariate OR,[0.025,0.975],p value,Multivariate OR,multi [0.025,multi 0.975],multi p value
age_at_index_date,1.00,1.00,1.00,1.000,1.00,0.99,1.00,0.000
antidepressants_100_pdds,1.03,1.03,1.04,0.000,1.02,1.01,1.02,0.000
benzo_and_z_drugs_100_pdds,1.02,1.02,1.03,0.000,1.01,1.00,1.01,0.000
chronic_pulmonary_disease,1.34,1.27,1.42,0.000,1.31,1.23,1.39,0.000
diabetes,1.31,1.20,1.42,0.000,1.26,1.15,1.38,0.000
epilepsy,1.68,1.37,2.07,0.000,1.50,1.21,1.85,0.000
hypertension,1.07,1.03,1.11,0.000,1.08,1.03,1.13,0.002
insomnia_count:1_5,1.49,1.33,1.67,0.000,1.21,1.08,1.37,0.001
insomnia_count:above_5,3.22,1.53,6.81,0.002,2.24,1.04,4.83,0.040
mental_illness_non_smi,1.62,1.55,1.69,0.000,1.78,1.69,1.88,0.000
